In [2]:
%reset -sf

In [317]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [172]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n\n")

In [160]:
srr = """
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
""".strip().split("\n\n")

In [173]:
def solve(arr, skip=(-1,-1)):

    n = len(arr)
    m = len(arr[0])
    
#     print(n, m)

    for i in range(1,n):
        for a,b in zip(arr[:i][::-1], arr[i:]):
            if a != b:
                break
        else:
            if (0, i) != skip:
                return (0, i)

    for j in range(1,m):
        flag = True
        for row in arr:
            for a,b in zip(row[:j][::-1], row[j:]):
                if a != b:
                    flag = False
        if flag:    
            if (j, 0) != skip:
                return (j, 0)
    
    return (-1, -1)

In [179]:
res = 0
res2 = 0
for brr in srr:
    arr = brr.strip().split("\n")
    arr = [list(row) for row in arr]
    a,b = solve(arr)
    
    n = len(arr)
    m = len(arr[0])
    
    
    for i in range(n):
        for j in range(m):
            if arr[i][j] == "#":
                arr[i][j] = "."
            else:
                arr[i][j] = "#"

            c,d = solve(arr, skip=(a,b))
            if (c,d) != (-1,-1):
                res2 += c
                res2 += 100*d
#                 print(c,d)
            
            if arr[i][j] == "#":
                arr[i][j] = "."
            else:
                arr[i][j] = "#"
    
    res += a
    res += 100 * b

In [180]:
res

31265

In [181]:
res2

78718

In [183]:
res2//2

39359